In [5]:
import pandas as pd

csv_file = "D:/Rokon/gtrs_hotels_2b547cba-9ee9-4bcd-a09e-806ae724df3f.csv"

# Load only the headers
df = pd.read_csv(csv_file, nrows=0)  # Read only the header row
headers = df.columns.tolist()


print(f"Headers saved to {headers}")



Headers saved to ['UnicaId', 'Name', 'Latitude', 'Longitude', 'AddressLine1', 'AddressLine2', 'CityName', 'StateName', 'StateCode', 'CountryName', 'CountryCode', 'PostalCode', 'Rating', 'PropertyType', 'ChainName', 'ChainCode', 'BrandName', 'CityCode', 'CityLocationId', 'MasterCityName', 'LocationIds', 'Phones', 'Emails', 'Fax', 'Website', 'AirportCodes', 'TrainStations', 'ProviderHotelId', 'ProviderFamily', 'GooglePlusCode', 'GooglePlaceId', 'Tags']


In [3]:
import pandas as pd

# Define the data types for the columns
dtype_dict = {
    'UnicaId': str,
    'ProviderHotelId': str,
    'ProviderFamily': str,
    'ModifiedOn': str,  
    'ChannelIds': str,
    'ProviderLocationCode': str
}

csv_file = "D:/Rokon/gtrs_mappings_2b547cba-9ee9-4bcd-a09e-806ae724df3f.csv"

# Load only the headers
df = pd.read_csv(csv_file, nrows=0)  # Read only the header row
headers = df.columns.tolist()

# Load only the first 100 rows with the specified data types
df = pd.read_csv(csv_file, dtype=dtype_dict, nrows=100)  # Load only the first 100 rows

print(df)
print(f"Headers saved to {headers}")


     UnicaId ProviderHotelId ProviderFamily              ModifiedOn  \
0   16188944         1000000          Agoda  10/11/2024 11:56:29 PM   
1   17008826        10000072          Agoda    5/16/2024 4:55:54 AM   
2   31069110         1000009          Agoda   10/11/2024 9:44:38 PM   
3   39654021          100001          Agoda    9/13/2024 1:53:29 AM   
4   39743166         1000020          Agoda   10/10/2024 2:43:10 AM   
..       ...             ...            ...                     ...   
95  15299150          100041          Agoda    5/12/2024 3:20:44 PM   
96  15687566         1000420          Agoda   10/11/2024 2:42:05 PM   
97  32177111        10004220          Agoda   8/24/2024 11:31:32 AM   
98  15614587         1000422          Agoda   10/25/2023 6:42:27 AM   
99  39628456         1000423          Agoda    10/8/2024 7:21:24 PM   

   ChannelIds ProviderLocationCode  
0         NaN                  NaN  
1         NaN                  NaN  
2         NaN                  NaN  

In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

csv_file = "D:/Rokon/gtrs_mappings_2b547cba-9ee9-4bcd-a09e-806ae724df3f.csv"

# Read the file with specific memory limits
df = pd.read_csv(csv_file, dtype=dtype_dict, low_memory=False)

print(df.head())  # Check the first few rows to see if the file was read properly


    UnicaId ProviderHotelId ProviderFamily              ModifiedOn ChannelIds  \
0  16188944         1000000          Agoda  10/11/2024 11:56:29 PM        NaN   
1  17008826        10000072          Agoda    5/16/2024 4:55:54 AM        NaN   
2  31069110         1000009          Agoda   10/11/2024 9:44:38 PM        NaN   
3  39654021          100001          Agoda    9/13/2024 1:53:29 AM        NaN   
4  39743166         1000020          Agoda   10/10/2024 2:43:10 AM        NaN   

  ProviderLocationCode  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN  


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

dtype_dict = {
    'UnicaId': str,
    'ProviderHotelId': str,
    'ProviderFamily': str,
    'ModifiedOn': str,  
    'ChannelIds': str,
    'ProviderLocationCode': str
}

csv_file = "D:/Rokon/gtrs_mappings_2b547cba-9ee9-4bcd-a09e-806ae724df3f.csv"

df = pd.read_csv(csv_file, dtype=dtype_dict)

# Convert ModifiedOn to datetime
df['ModifiedOn'] = pd.to_datetime(df['ModifiedOn'], format='%m/%d/%Y %I:%M:%S %p')

# Fill NaN values - Set string columns to None for SQL NULL representation
df = df.where(pd.notnull(df), None)

header = df.columns.tolist()
print("Header:", header)
# print("Length of header:", len(header))
# print(df)  

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  
        database="innova_db_v1.25"
    )

    if connection.is_connected():
        cursor = connection.cursor()

        # Create the info_01 table if it doesn't exist
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS gtrs_mappings (
            UnicaId VARCHAR(255),
            ProviderHotelId VARCHAR(255),
            ProviderFamily VARCHAR(255),
            ModifiedOn DATETIME,
            ChannelIds VARCHAR(255),
            ProviderLocationCode VARCHAR(255)
        );
        '''

        cursor.execute(create_table_query)

        # Prepare the insert statement for the info_01 table
        insert_query = '''INSERT INTO gtrs_mappings (UnicaId, ProviderHotelId, ProviderFamily, ModifiedOn, ChannelIds, ProviderLocationCode)
                          VALUES (%s, %s, %s, %s, %s, %s)'''

        num = 0
        # Iterate over DataFrame rows and insert data
        for i, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))
            connection.commit()  
            print(f"{num}: Successfully loaded.")
            num += 1

        print("Data inserted successfully")
except Error as e:
    print(f"Error while connecting to MySql: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySql connection closed")
